In [ ]:
from os.path import join

import numpy as np
import cupy as xp 
import random

import librosa
from scipy import signal
from features_toolkit import get_csv_labels, find_longest_file, features_extraction, coeffs_create, coeffs_load

from svm import SVM as svm_gpu

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay
from sklearn.model_selection import KFold
import optuna

# MFCC coefficients optimization

In [ ]:
def objective(trial):
    DATA_PATH = 'spcup_2022_training_part1'
    CSV_PATH = join(DATA_PATH, 'labels.csv')
    y = get_csv_labels(CSV_PATH)

    longest_file, longest_file_path = find_longest_file(DATA_PATH)
    longest_file_size = longest_file.shape[0]

    vals = [256, 512, 1024, 2048]
    n_fft = trial.suggest_categorical('n_fft', vals)
    win_length = round(n_fft*trial.suggest_categorical('win_length', [0.25, 0.5, 0.75, 1]))
    win_hop = trial.suggest_categorical('win_hop', [0.375, 0.5, 0.625, 0.750, 0.875])
    hop_length = round(win_length*win_hop)

    n_mels = trial.suggest_categorical('n_mels', [32, 64, 128, 256])
    n_mfcc = round(n_mels*float(trial.suggest_categorical('n_mfcc', 1/np.arange(1,32))))

    MFCC_kwargs = {
          #mfcc kwargs
          'n_mfcc':n_mfcc,
          'dct_type':trial.suggest_int('dct_type', 1, 3),
          'norm':trial.suggest_categorical('norm', [None, 'ortho']),
          'lifter':trial.suggest_float('lifter', 0.1, 3),
          'delta':trial.suggest_int('delta', 0, 5),
          #stft kwargs
          'n_fft':n_fft, 
          'win_length':win_length,
          'hop_length':hop_length,
          'window':'hann',
          #filer kwargs
          'n_mels':n_mels,
          'fmin':0.0, # 0-8000
          'fmax':None}# 0-8000
    display(MFCC_kwargs)
    
    X = features_extraction(CSV_PATH=CSV_PATH, DATA_PATH=DATA_PATH, features_type="MFCC", windowing=True, longest_file_size=longest_file_size, **MFCC_kwargs)
    
    clf = svm_gpu(kernel='linear', kernel_params={}, classification_strategy='ovr', max_iter=200, x=xp.asarray(X), y=xp.asarray(y), n_folds=5, lambduh=1.159)

    accuracy = 1-np.mean(clf.cross_validation_error())

    return accuracy


study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=50)

[I 2022-02-19 20:46:04,753] A new study created in memory with name: no-name-8eaf72b4-c6d5-416d-a9fa-1db135d51968


{'dct_type': 2,
 'delta': 3,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 128,
 'lifter': 1.8560463604420145,
 'n_fft': 256,
 'n_mels': 128,
 'n_mfcc': 6,
 'norm': 'ortho',
 'win_length': 256,
 'window': 'hann'}

100%|██████████| 5000/5000 [01:00<00:00, 82.58it/s]
[I 2022-02-19 20:49:13,568] Trial 0 finished with value: 0.4188338227413996 and parameters: {'n_fft': '256', 'win_length': '1', 'win_hop': '0.5', 'n_mels': 128, 'n_mfcc': 0.047619047619047616, 'dct_type': 2, 'norm': 'ortho', 'lifter': 1.8560463604420145, 'delta': 3}. Best is trial 0 with value: 0.4188338227413996.


{'dct_type': 3,
 'delta': 5,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 192,
 'lifter': 1.6476606187460814,
 'n_fft': 1024,
 'n_mels': 64,
 'n_mfcc': 32,
 'norm': None,
 'win_length': 256,
 'window': 'hann'}

100%|██████████| 5000/5000 [01:09<00:00, 71.91it/s]
[I 2022-02-19 20:53:20,342] Trial 1 finished with value: 0.5570544028864359 and parameters: {'n_fft': '1024', 'win_length': '0.25', 'win_hop': '0.750', 'n_mels': 64, 'n_mfcc': 0.5, 'dct_type': 3, 'norm': None, 'lifter': 1.6476606187460814, 'delta': 5}. Best is trial 1 with value: 0.5570544028864359.


...

100%|██████████| 5000/5000 [00:40<00:00, 123.39it/s]
[I 2022-02-19 23:29:49,695] Trial 47 finished with value: 0.5219752688172044 and parameters: {'n_fft': '2048', 'win_length': '0.75', 'win_hop': '0.875', 'n_mels': 64, 'n_mfcc': 0.16666666666666666, 'dct_type': 3, 'norm': None, 'lifter': 1.2935612569637827, 'delta': 1}. Best is trial 14 with value: 0.7200936091165648.


{'dct_type': 3,
 'delta': 2,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 384,
 'lifter': 1.5748223172587847,
 'n_fft': 1024,
 'n_mels': 256,
 'n_mfcc': 64,
 'norm': None,
 'win_length': 768,
 'window': 'hann'}

100%|██████████| 5000/5000 [01:14<00:00, 67.06it/s]
[I 2022-02-19 23:33:24,249] Trial 48 finished with value: 0.658967309684564 and parameters: {'n_fft': '1024', 'win_length': '0.75', 'win_hop': '0.5', 'n_mels': 256, 'n_mfcc': 0.25, 'dct_type': 3, 'norm': None, 'lifter': 1.5748223172587847, 'delta': 2}. Best is trial 14 with value: 0.7200936091165648.


{'dct_type': 2,
 'delta': 4,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 160,
 'lifter': 0.9212744032257788,
 'n_fft': 512,
 'n_mels': 256,
 'n_mfcc': 16,
 'norm': None,
 'win_length': 256,
 'window': 'hann'}

100%|██████████| 5000/5000 [01:23<00:00, 59.61it/s]
[I 2022-02-19 23:37:09,057] Trial 49 finished with value: 0.5594186618083092 and parameters: {'n_fft': '512', 'win_length': '0.5', 'win_hop': '0.625', 'n_mels': 256, 'n_mfcc': 0.0625, 'dct_type': 2, 'norm': None, 'lifter': 0.9212744032257788, 'delta': 4}. Best is trial 14 with value: 0.7200936091165648.


# LFCC coefficients optimization

In [ ]:
def objective(trial):
    DATA_PATH = 'spcup_2022_training_part1'
    CSV_PATH = join(DATA_PATH, 'labels.csv')
    y = get_csv_labels(CSV_PATH)

    longest_file, longest_file_path = find_longest_file(DATA_PATH)
    longest_file_size = longest_file.shape[0]

    vals = [256, 512, 1024, 2048]
    n_fft = trial.suggest_categorical('n_fft', vals)
    win_length = round(n_fft*trial.suggest_categorical('win_length', [0.25, 0.5, 0.75, 1]))
    win_hop = trial.suggest_categorical('win_hop', [0.375, 0.5, 0.625, 0.750, 0.875])
    hop_length = round(win_length*win_hop)

    n_filter = trial.suggest_categorical('n_filter', [32, 64, 128, 256])
    n_lfcc = round(n_filter*trial.suggest_categorical('n_lfcc', 1/np.arange(1,32)))

    LFCC_kwargs = {
          #lfcc kwargs
          'S':None, # Spect matrix, if none it will be calculated
          'n_lfcc':n_lfcc,
          'dct_type':trial.suggest_int('dct_type', 1, 4),
          'norm':trial.suggest_categorical('norm', [None, 'ortho']),
          'delta':trial.suggest_int('delta', 0, 5),
          #stft kwargs
          'n_fft':n_fft, 
          'win_length':win_length,
          'hop_length':hop_length,
          'window':'hann',
          #filer kwargs
          'n_filter':n_filter,
          'fmin':0.0, # 0-8000
          'fmax':None}# 0-8000
    display(LFCC_kwargs)
    
    X = features_extraction(CSV_PATH=CSV_PATH, DATA_PATH=DATA_PATH, features_type="LFCC", windowing=True, longest_file_size=longest_file_size, **LFCC_kwargs)

    clf = svm_gpu(kernel='linear', kernel_params={}, classification_strategy='ovr', max_iter=200, x=xp.asarray(X), y=xp.asarray(y), n_folds=5, lambduh=1.159)

    accuracy = 1-np.mean(clf.cross_validation_error())

    return accuracy


study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=50)

[I 2022-02-20 02:48:02,172] A new study created in memory with name: no-name-54f7ebc0-b627-4ca9-bdbb-56754a877930


{'S': None,
 'dct_type': 3,
 'delta': 0,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 144,
 'n_fft': 256,
 'n_filter': 64,
 'n_lfcc': 3,
 'norm': 'ortho',
 'win_length': 192,
 'window': 'hann'}

100%|██████████| 5000/5000 [00:31<00:00, 160.65it/s]
[I 2022-02-20 02:51:44,112] Trial 0 finished with value: 0.3152413782016934 and parameters: {'n_fft': 256, 'win_length': 0.75, 'win_hop': 0.75, 'n_filter': 64, 'n_lfcc': 0.045454545454545456, 'dct_type': 3, 'norm': 'ortho', 'delta': 0}. Best is trial 0 with value: 0.3152413782016934.


{'S': None,
 'dct_type': 3,
 'delta': 3,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 168,
 'n_fft': 256,
 'n_filter': 32,
 'n_lfcc': 2,
 'norm': 'ortho',
 'win_length': 192,
 'window': 'hann'}

100%|██████████| 5000/5000 [00:27<00:00, 180.34it/s]
[I 2022-02-20 02:54:01,947] Trial 1 finished with value: 0.2634740470831993 and parameters: {'n_fft': 256, 'win_length': 0.75, 'win_hop': 0.875, 'n_filter': 32, 'n_lfcc': 0.07692307692307693, 'dct_type': 3, 'norm': 'ortho', 'delta': 3}. Best is trial 0 with value: 0.3152413782016934.


...

100%|██████████| 5000/5000 [00:45<00:00, 110.39it/s]
[I 2022-02-20 05:19:41,409] Trial 47 finished with value: 0.6401278444611154 and parameters: {'n_fft': 512, 'win_length': 0.75, 'win_hop': 0.625, 'n_filter': 128, 'n_lfcc': 0.14285714285714285, 'dct_type': 1, 'norm': None, 'delta': 2}. Best is trial 24 with value: 0.7124349855320973.


{'S': None,
 'dct_type': 3,
 'delta': 3,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 320,
 'n_fft': 1024,
 'n_filter': 128,
 'n_lfcc': 5,
 'norm': None,
 'win_length': 512,
 'window': 'hann'}

100%|██████████| 5000/5000 [00:47<00:00, 104.96it/s]
[I 2022-02-20 05:22:18,860] Trial 48 finished with value: 0.37998983317257895 and parameters: {'n_fft': 1024, 'win_length': 0.5, 'win_hop': 0.625, 'n_filter': 128, 'n_lfcc': 0.03571428571428571, 'dct_type': 3, 'norm': None, 'delta': 3}. Best is trial 24 with value: 0.7124349855320973.


{'S': None,
 'dct_type': 2,
 'delta': 1,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 96,
 'n_fft': 512,
 'n_filter': 256,
 'n_lfcc': 12,
 'norm': None,
 'win_length': 256,
 'window': 'hann'}

100%|██████████| 5000/5000 [01:13<00:00, 67.70it/s]
[I 2022-02-20 05:25:28,857] Trial 49 finished with value: 0.6196594487907691 and parameters: {'n_fft': 512, 'win_length': 0.5, 'win_hop': 0.375, 'n_filter': 256, 'n_lfcc': 0.045454545454545456, 'dct_type': 2, 'norm': None, 'delta': 1}. Best is trial 24 with value: 0.7124349855320973.


# CQCC coefficients optimization

In [ ]:
def objective(trial):
    DATA_PATH = 'spcup_2022_training_part1'
    CSV_PATH = join(DATA_PATH, 'labels.csv')
    y = get_csv_labels(CSV_PATH)

    longest_file, longest_file_path = find_longest_file(DATA_PATH)
    longest_file_size = longest_file.shape[0]

    BPO, n_bins = trial.suggest_categorical('BPO_n_bins', [(3,25), (6,50), (9,75), (12,100)])

    CQCC_kwargs = {
          'fmin':10, 
          'n_bins':n_bins,
          'BPO':BPO,  
          'n_coef':trial.suggest_int('n_coef', 5, 100),
          'TBL':512, 
          'delta':trial.suggest_int('delta', 0, 5),
          }
    display(CQCC_kwargs)
    
    X = features_extraction(CSV_PATH=CSV_PATH, features_type="CQCC", DATA_PATH=DATA_PATH, windowing=True, longest_file_size=longest_file_size, **CQCC_kwargs)
    
    clf = svm_gpu(kernel='linear', kernel_params={}, classification_strategy='ovr', max_iter=200, x=xp.asarray(X), y=xp.asarray(y), n_folds=5, lambduh=1.159)

    accuracy = 1-np.mean(clf.cross_validation_error())

    return accuracy


study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=20)

[I 2022-02-20 05:25:28,927] A new study created in memory with name: no-name-64f62fc7-2782-403c-a984-68820d198bcc


{'BPO': 3, 'TBL': 512, 'delta': 4, 'fmin': 10, 'n_bins': 25, 'n_coef': 25}

100%|██████████| 5000/5000 [06:35<00:00, 12.64it/s]
[I 2022-02-20 05:34:46,128] Trial 0 finished with value: 0.3347525684992677 and parameters: {'BPO_n_bins': (3, 25), 'n_coef': 25, 'delta': 4}. Best is trial 0 with value: 0.3347525684992677.


{'BPO': 6, 'TBL': 512, 'delta': 0, 'fmin': 10, 'n_bins': 50, 'n_coef': 76}

100%|██████████| 5000/5000 [11:31<00:00,  7.23it/s]
[I 2022-02-20 05:48:36,156] Trial 1 finished with value: 0.4727656324795484 and parameters: {'BPO_n_bins': (6, 50), 'n_coef': 76, 'delta': 0}. Best is trial 1 with value: 0.4727656324795484.


{'BPO': 12, 'TBL': 512, 'delta': 2, 'fmin': 10, 'n_bins': 100, 'n_coef': 30}

100%|██████████| 5000/5000 [23:17<00:00,  3.58it/s]
[I 2022-02-20 06:14:15,392] Trial 2 finished with value: 0.41057485621405354 and parameters: {'BPO_n_bins': (12, 100), 'n_coef': 30, 'delta': 2}. Best is trial 1 with value: 0.4727656324795484.


{'BPO': 12, 'TBL': 512, 'delta': 4, 'fmin': 10, 'n_bins': 100, 'n_coef': 73}

 38%|███▊      | 1903/5000 [09:21<18:08,  2.85it/s]

# Best coefficients params

In [ ]:
# from cqcc import cqcc
best_MFCC_params = {'dct_type': 2,
 'delta': 2,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 128,
 'lifter': 2.33,
 'n_fft': 512,
 'n_mels': 256,
 'n_mfcc': 43,
 'norm': None,
 'win_length': 256,
 'window': 'hann'}
best_LFCC_params = {'S': None,
 'dct_type': 2,
 'delta': 2,
 'fmax': None,
 'fmin': 0.0,
 'hop_length': 160,
 'n_fft': 512,
 'n_filter': 128,
 'n_lfcc': 128,
 'norm': None,
 'win_length': 256,
 'window': 'hann'}
best_CQCC_params = {'BPO': 6, 
  'TBL': 512, 
  'delta': 2,   
  'fmin': 10, 
  'n_bins': 50, 
  'n_coef': 20}

DATA_PATH = 'spcup_2022_training_part1'
CSV_PATH = join(DATA_PATH, 'labels.csv')
y = get_csv_labels(CSV_PATH)

coeffs_create(CSV_PATH, DATA_PATH, best_MFCC_params, best_LFCC_params, best_CQCC_params)

'MFCC'

100%|██████████| 5000/5000 [01:14<00:00, 67.26it/s]


'LFCC'

100%|██████████| 5000/5000 [00:49<00:00, 101.71it/s]


'CQCC'

...

100%|██████████| 5000/5000 [01:27<00:00, 57.40it/s]


'ILFCC'

100%|██████████| 5000/5000 [02:13<00:00, 37.47it/s]


'ICQCC'

100%|██████████| 5000/5000 [11:11<00:00,  7.45it/s]


# SVM with linear kernel optimization

In [ ]:
def objective(trial):

    CSV_PATH = join(DATA_PATH, 'labels.csv')
    y = get_csv_labels(CSV_PATH)

    coeff_type = trial.suggest_categorical('coeff_type', ['MFCC', 'LFCC', 'CQCC', 'IMFCC', 'ILFCC', 'ICQCC'])

    SVM_params = {
          'lambduh':trial.suggest_float('lambduh', 0.01, 10),
          # 'max_iter':trial.suggest_categorical('max_iter', np.array([200, 300, 400])),
          'classification_strategy':trial.suggest_categorical('classification_strategy', ['ovo', 'ovr'])
          } 
    display(SVM_params, coeff_type)
    
    X = coeffs_load(coeff_type=coeff_type)

    clf = svm_gpu(kernel='linear', kernel_params={}, max_iter=200, x=xp.asarray(X), y=xp.asarray(y), n_folds=5, **SVM_params)

    accuracy = 1-np.mean(clf.cross_validation_error())

    return accuracy


study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=50)

[I 2022-02-20 16:13:51,470] A new study created in memory with name: no-name-d3cbc315-1558-4f2c-87d3-cc5dd30391d2


{'classification_strategy': 'ovo', 'lambduh': 2.9111368862696225}

'LFCC'

[I 2022-02-20 16:23:24,618] Trial 0 finished with value: 0.7375898867574037 and parameters: {'coeff_type': 'LFCC', 'lambduh': 2.9111368862696225, 'classification_strategy': 'ovo'}. Best is trial 0 with value: 0.7375898867574037.


{'classification_strategy': 'ovr', 'lambduh': 3.8932857832497483}

...

'MFCC'

[I 2022-02-20 20:44:10,941] Trial 48 finished with value: 0.7565437323616618 and parameters: {'coeff_type': 'MFCC', 'lambduh': 1.2260707863099078, 'classification_strategy': 'ovo'}. Best is trial 7 with value: 0.7565437323616618.


{'classification_strategy': 'ovr', 'lambduh': 3.4372301127951426}

'ILFCC'

[I 2022-02-20 20:48:07,097] Trial 49 finished with value: 0.20003990997749443 and parameters: {'coeff_type': 'ILFCC', 'lambduh': 3.4372301127951426, 'classification_strategy': 'ovr'}. Best is trial 7 with value: 0.7565437323616618.


# SVM with sigmoid kernel optimization

In [ ]:
def objective(trial):

    CSV_PATH = join(DATA_PATH, 'labels.csv')
    y = get_csv_labels(CSV_PATH)

    coeff_type = trial.suggest_categorical('coeff_type', ['MFCC', 'LFCC', 'CQCC', 'IMFCC', 'ILFCC', 'ICQCC'])

    SVM_params = {
          'lambduh':trial.suggest_float('lambduh', 0.01, 10),
          # 'max_iter':trial.suggest_int('lambduh', 100, 1000),
          'classification_strategy':trial.suggest_categorical('classification_strategy', ['ovo', 'ovr'])
          } 

    kernel_params = {
          'alpha':trial.suggest_float('lambduh', -20, 20),
          'beta':trial.suggest_float('lambduh', -20, 20)
          }
    
    display(SVM_params, kernel_params, coeff_type)
    
    X = coeffs_load(coeff_type=coeff_type)

    clf = svm_gpu(kernel='sigmoid', kernel_params=kernel_params, max_iter=200, x=xp.asarray(X), y=xp.asarray(y), n_folds=5, **SVM_params)

    accuracy = 1-np.mean(clf.cross_validation_error())

    return accuracy


study = optuna.create_study(direction="maximize")

study.optimize(objective, n_trials=50)

[I 2022-02-20 20:48:08,840] A new study created in memory with name: no-name-fe4d15a5-9f7c-4abd-a19d-79bab89c12ec


{'classification_strategy': 'ovo', 'lambduh': 2.52830455555939}

{'alpha': 2.52830455555939, 'beta': 2.52830455555939}

'CQCC'

[I 2022-02-20 20:53:07,872] Trial 0 finished with value: 0.2301890168970815 and parameters: {'coeff_type': 'CQCC', 'lambduh': 2.52830455555939, 'classification_strategy': 'ovo'}. Best is trial 0 with value: 0.2301890168970815.


...

'LFCC'

[I 2022-02-20 22:05:00,806] Trial 14 finished with value: 0.28051569678133836 and parameters: {'coeff_type': 'LFCC', 'lambduh': 1.330095638295309, 'classification_strategy': 'ovo'}. Best is trial 12 with value: 0.2806228217768727.


{'classification_strategy': 'ovo', 'lambduh': 1.627283173768965}

{'alpha': 1.627283173768965, 'beta': 1.627283173768965}

'LFCC'